## Import Packages

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

## Load in Data

In [2]:
data = pd.read_csv("cobra-clean.csv", sep=',', header=0, engine='python', usecols = lambda column : column in ["Occur Time", "Latitude", "Longitude"])
data.head()

print(data)

        Occur Time  Longitude  Latitude
0             1145  -84.49328  33.68845
1             1330  -84.39201  33.75320
2             1450  -84.36212  33.84676
3             1600  -84.49773  33.68677
4             1645  -84.34875  33.75786
5             1740  -84.45385  33.70827
6             1627  -84.36212  33.84676
7              815  -84.48742  33.68841
8             2000  -84.39702  33.76269
9             2300  -84.33120  33.75902
10            2230  -84.32465  33.75441
11             100  -84.43612  33.71464
12            2030  -84.37944  33.85115
13             130  -84.34595  33.76675
14             900  -84.36687  33.75283
15            1915  -84.37334  33.78080
16            1900  -84.38815  33.77023
17            1700  -84.38139  33.78663
18            1700  -84.36851  33.77917
19             100  -84.47023  33.79988
20            1400  -84.39534  33.76464
21            1630  -84.39202  33.87589
22            1200  -84.38730  33.77754
23            2000  -84.49773  33.68677


In [3]:
from sklearn.tree import plot_tree
from sklearn.tree.export import export_text

# Read the data for the training features
dataX = pd.read_csv("cobra-clean.csv", sep=',', header=0, engine='python', usecols = lambda column : column in ["Neighborhood", "Month", "Day", "Shift"])
dataX.head()
X = dataX
# Read the data for the training label
dataY = pd.read_csv("cobra-clean.csv", sep=',', header=0, engine='python', usecols = lambda column : column in ["Crime Score"])
dataY.head()
Y = dataY
# Train
clf = DecisionTreeClassifier()
clf = clf.fit(X, Y)
# Read the data for the test features
dataZ = pd.read_csv("cobra-2019.csv", sep=',', header=0, engine='python', usecols = lambda column : column in ["Neighborhood", "Month", "Day", "Shift"])
dataZ.head()
Z = dataZ
# Read the data for the test label
dataW = pd.read_csv("cobra-2019.csv", sep=',', header=0, engine='python', usecols = lambda column : column in ["Crime Score"])
dataW.head()
W = dataW
# Predict based on the given features for the test set
predictions = clf.predict(Z)
predictions_probability = clf.predict_proba(Z)
# Compare to the actual labels
'''Insert comparison methods here'''
# Plot the tree
plot_tree(clf.fit(X, iris.target))

# Export a text file containing the rules for the decision tree
'''decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2) # 
decision_tree = decision_tree.fit(X, Y)
r = export_text(decision_tree, feature_names=iris['feature_names'])
print(r)'''

ValueError: could not convert string to float: 'Greenbriar'